In [ ]:
!wget https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt

--2021-10-26 04:19:17--  https://github.com/MorvanZhou/PyTorch-Tutorial/raw/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt [following]
--2021-10-26 04:19:18--  https://raw.githubusercontent.com/MorvanZhou/PyTorch-Tutorial/master/tutorial-contents-notebooks/mnist/processed/training.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47520431 (45M) [application/octet-stream]
Saving to: ‘training.pt’

training.pt         100%[===================>]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import tqdm

In [ ]:
def normalise_x(x):
    return (x - x.min())/(x.max() - x.min())

In [ ]:
X_train, y_train = torch.load('training.pt')
X_train = X_train.reshape(-1,28*28)
X_train = normalise_x(X_train)
train_tensor_data = TensorDataset(X_train.float(), X_train.float())
train_dl = DataLoader(train_tensor_data, batch_size=128, shuffle=True)

In [ ]:
device = torch.device('cuda')

In [ ]:
class DNN_Autoencoder(nn.Module):

    def __init__(self):

        super(DNN_Autoencoder, self).__init__()

        self.enc_fc1 = nn.Linear(in_features = 28*28, out_features = 128)
        self.enc_fc2 = nn.Linear(in_features = 128, out_features = 28)
        self.dec_fc1 = nn.Linear(in_features = 28, out_features = 128)
        self.dec_fc2 = nn.Linear(in_features = 128, out_features = 28*28)

    def encoder(self, x):

        x = self.enc_fc1(x)
        x = F.relu(x)
        x = self.enc_fc2(x)

        return x
    
    def decoder(self, x):

        x = self.dec_fc1(x)
        x = F.relu(x)
        x = self.dec_fc2(x)

        return x

    def forward(self, x):

        x = self.encoder(x)
        x = F.relu(x)
        x = self.decoder(x)

        return x

In [ ]:
AE = DNN_Autoencoder()
AE = AE.to(device)
opt = Adam(params = AE.parameters(), lr=1e-2)

In [ ]:
def trainer(epochs):

    for epoch in range(epochs):

        training_loss = 0.0

        for batch, target in train_dl:

            batch = batch.to(device)
            target = target.to(device)

            preds = AE(batch)

            loss = F.mse_loss(preds, target)
            training_loss += loss.item()

            loss.backward()
            opt.step()
            opt.zero_grad()

        if epoch%(epochs//10) == 0:
            print(f"Epoch: {epoch}    Training loss: {training_loss}")

In [ ]:
trainer(100)

Epoch: 0    Training loss: 17.580722626298666
Epoch: 10    Training loss: 13.134700046852231
Epoch: 20    Training loss: 13.005562519654632
Epoch: 30    Training loss: 12.93203934840858
Epoch: 40    Training loss: 12.899282108992338


In [ ]:
plt.imshow(X_train[300].reshape(28,28))

In [ ]:
plt.imshow(AE(X_train[300].float().to(device)).detach().cpu().numpy().reshape(28,28))

In [ ]:
from sklearn.manifold import TSNE
t =TSNE()

In [ ]:
data_reduced = AE.encoder(X_train.float().to(device))
tsne_mat = t.fit_transform(data_reduced.detach().cpu().numpy())

In [ ]:
plt.scatter(tsne_mat[:,0], tsne_mat[:,1], c=y_train.detach().cpu().numpy())